In [1]:
# install required libraries
!pip3 install sentence-transformers

     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 2.0MB 26.8MB/s 
     |████████████████████████████████| 1.2MB 49.3MB/s 
     |████████████████████████████████| 890kB 52.1MB/s 
     |████████████████████████████████| 3.2MB 51.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.3-cp37-none-any.whl size=114277 sha256=b5524e0351c87beb2af5e0e636d17165b3b19ec9e113c892a6e880a24b6c9d70
  Stored in directory: /root/.cache/pip/wheels/c8/0d/fa/4e28cf045da4781344e7972befb2fdf306051b225bfc290187
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=992aec4c4c229ea83ddb4681e7a30432bd4c8d71e7b40b511999cb508ab3c677
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
!pip3 install elasticsearch

     |████████████████████████████████| 337kB 18.0MB/s 


In [3]:
from google.colab import drive

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/BERT-FAQ

/content/drive/MyDrive/BERT-FAQ


In [7]:
!ls

data			     metric.py	       reranker.py
evaluation.py		     notebook	       searcher.py
faq_bert_finetuning.py	     output	       shared
faq_bert.py		     parser	       training_data_generator.py
faq_bert_ranker.py	     __pycache__       webserver.py
hard_negatives_generator.py  README.md
indexer.py		     requirements.txt


In [8]:
# import required dependencies
from evaluation import get_relevance_label_df
from shared.utils import load_from_json
from shared.utils import dump_to_json
from shared.utils import make_dirs
from reranker import ReRanker

In [9]:
output_path="data/FAQIR/rank_results"

# load user_query ES results from json files
es_output_path = output_path + "/unsupervised"
es_query_by_question = load_from_json(es_output_path + '/es_query_by_question.json')
es_query_by_answer = load_from_json(es_output_path + '/es_query_by_answer.json')
es_query_by_question_answer = load_from_json(es_output_path + '/es_query_by_question_answer.json')
es_query_by_question_answer_concat = load_from_json(es_output_path + '/es_query_by_question_answer_concat.json')

In [10]:
# load test_queries, relevance_label_df for ReRanker
query_answer_pair_filepath = 'data/FAQIR/query_answer_pairs.json'
relevance_label_df = get_relevance_label_df(query_answer_pair_filepath)
test_queries = relevance_label_df[relevance_label_df['query_type'] == 'user_query'].question.unique()

In [11]:
test_queries[:10]

array(['How to increase the efficiency of my car with respect to fuel expended?',
       'Is there a special way to drive to decreasse my fuel consumption?',
       'How to use less gas in my car?',
       'I want to save some money on gas. What can I do to make my car use less gas?',
       'Are there any simple tweaks one can do to a car to make it more fuel efficient?',
       'How can I make my fuel mileage bigger?',
       'I feel my car is using too much gas. Can I make some cheap modifications that would fix this?',
       'What should I do to reduce the fuel required by my car while not affecting the mileage?',
       'I need to decrease the fuel consumption of my car. What are the best strategies?',
       'I have a sporty style of driving, can I reduce the fuel consumption by changing the driving style?'],
      dtype=object)

In [12]:
# total number of test queries
len(test_queries)

1183

**#################### Triplet Loss ######################**

**Re-ranking results using query_type="user_query"; neg_type="hard"**

In [12]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [13]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [14]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-23 17:32:10 - Generating BERT top-k results ...
2021-03-23 17:32:11 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_hard_user_query_1.1
2021-03-23 17:32:11 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_hard_user_query_1.1
2021-03-23 17:32:19 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:58<00:00,  1.23it/s]
2021-03-23 17:48:20 - Re-ranking the top-k results ...
2021-03-23 17:48:22 - Generating BERT top-k results ...
2021-03-23 17:48:22 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_hard_user_query_1.1
2021-03-23 17:48:22 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_hard_user_query_1.1
2021-03-23 17:48:24 - Use pytorch device: cuda
100%|██████████| 1183/1183 [17:13<00:00,  1.14it/s]
2021-03-23 18:05:40 - Re-ranking the top-k results ...
2021-03-23 18:05:42 - Generating BERT top-k results ...
2021-03-23 18:05:42 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_hard_us

**Re-ranking results using query_type="user_query"; neg_type="simple"**

In [15]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [16]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [17]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-23 18:38:44 - Generating BERT top-k results ...
2021-03-23 18:38:44 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_simple_user_query_1.1
2021-03-23 18:38:44 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_simple_user_query_1.1
2021-03-23 18:38:55 - Use pytorch device: cuda
100%|██████████| 1183/1183 [18:54<00:00,  1.04it/s]
2021-03-23 18:57:52 - Re-ranking the top-k results ...
2021-03-23 18:57:54 - Generating BERT top-k results ...
2021-03-23 18:57:54 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_simple_user_query_1.1
2021-03-23 18:57:54 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_simple_user_query_1.1
2021-03-23 18:57:56 - Use pytorch device: cuda
100%|██████████| 1183/1183 [24:09<00:00,  1.23s/it]
2021-03-23 19:22:08 - Re-ranking the top-k results ...
2021-03-23 19:22:11 - Generating BERT top-k results ...
2021-03-23 19:22:11 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet

**Re-ranking results using query_type="faq"; neg_type="hard"**

In [18]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [19]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [20]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-23 20:05:32 - Generating BERT top-k results ...
2021-03-23 20:05:32 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_hard_faq_1.1
2021-03-23 20:05:32 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_hard_faq_1.1
2021-03-23 20:05:42 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:32<00:00,  1.27it/s]
2021-03-23 20:21:17 - Re-ranking the top-k results ...
2021-03-23 20:21:19 - Generating BERT top-k results ...
2021-03-23 20:21:19 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_hard_faq_1.1
2021-03-23 20:21:19 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_hard_faq_1.1
2021-03-23 20:21:20 - Use pytorch device: cuda
100%|██████████| 1183/1183 [17:03<00:00,  1.16it/s]
2021-03-23 20:38:27 - Re-ranking the top-k results ...
2021-03-23 20:38:29 - Generating BERT top-k results ...
2021-03-23 20:38:29 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_hard_faq_1.1
2021-03-23 20:38:29 - 

**Re-ranking results using query_type="faq"; neg_type="simple"**

In [21]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='triplet'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [22]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [23]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-23 21:11:24 - Generating BERT top-k results ...
2021-03-23 21:11:24 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_simple_faq_1.1
2021-03-23 21:11:24 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_simple_faq_1.1
2021-03-23 21:11:35 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:31<00:00,  1.27it/s]
2021-03-23 21:27:08 - Re-ranking the top-k results ...
2021-03-23 21:27:10 - Generating BERT top-k results ...
2021-03-23 21:27:10 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_simple_faq_1.1
2021-03-23 21:27:10 - Load SentenceTransformer from folder: output/FAQIR/models/triplet_simple_faq_1.1
2021-03-23 21:27:12 - Use pytorch device: cuda
100%|██████████| 1183/1183 [17:05<00:00,  1.15it/s]
2021-03-23 21:44:20 - Re-ranking the top-k results ...
2021-03-23 21:44:22 - Generating BERT top-k results ...
2021-03-23 21:44:22 - Load pretrained SentenceTransformer: output/FAQIR/models/triplet_simple_faq_1.1
2021-03-23 2

**##################### Softmax Loss #####################**

**Re-ranking results using query_type="user_query"; neg_type="hard"**

In [24]:
# define variables
query_type="user_query"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [25]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [26]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-23 22:17:16 - Generating BERT top-k results ...
2021-03-23 22:17:27 - Use pytorch device: cuda
100%|██████████| 1183/1183 [14:37<00:00,  1.35it/s]
2021-03-23 22:32:07 - Re-ranking the top-k results ...
2021-03-23 22:32:09 - Generating BERT top-k results ...
2021-03-23 22:32:12 - Use pytorch device: cuda
100%|██████████| 1183/1183 [16:39<00:00,  1.18it/s]
2021-03-23 22:48:54 - Re-ranking the top-k results ...
2021-03-23 22:48:56 - Generating BERT top-k results ...
2021-03-23 22:48:58 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:54<00:00,  1.24it/s]
2021-03-23 23:04:55 - Re-ranking the top-k results ...
2021-03-23 23:04:58 - Generating BERT top-k results ...
2021-03-23 23:04:59 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:31<00:00,  1.27it/s]
2021-03-23 23:20:34 - Re-ranking the top-k results ...


**Re-ranking results using query_type="user_query"; neg_type="simple"**

In [13]:
# define variables
query_type="user_query"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [14]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [ ]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-23 23:20:37 - Generating BERT top-k results ...
2021-03-23 23:20:49 - Use pytorch device: cuda
100%|██████████| 1183/1183 [14:41<00:00,  1.34it/s]
2021-03-23 23:35:33 - Re-ranking the top-k results ...
2021-03-23 23:35:35 - Generating BERT top-k results ...
2021-03-23 23:35:37 - Use pytorch device: cuda
100%|██████████| 1183/1183 [16:42<00:00,  1.18it/s]
2021-03-23 23:52:23 - Re-ranking the top-k results ...
2021-03-23 23:52:26 - Generating BERT top-k results ...
2021-03-23 23:52:27 - Use pytorch device: cuda
 87%|████████▋ | 1026/1183 [13:47<02:08,  1.22it/s]

In [15]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

In [16]:
bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-24 05:39:49 - Generating BERT top-k results ...
2021-03-24 05:39:58 - Use pytorch device: cuda
100%|██████████| 1183/1183 [16:06<00:00,  1.22it/s]
2021-03-24 05:56:07 - Re-ranking the top-k results ...
2021-03-24 05:56:09 - Generating BERT top-k results ...
2021-03-24 05:56:11 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:42<00:00,  1.26it/s]
2021-03-24 06:11:56 - Re-ranking the top-k results ...


**Re-ranking results using query_type="faq"; neg_type="hard"**

In [17]:
# define variables
query_type="faq"; neg_type="hard"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [18]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [19]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')
bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-24 06:11:59 - Generating BERT top-k results ...
2021-03-24 06:12:10 - Use pytorch device: cuda
100%|██████████| 1183/1183 [14:51<00:00,  1.33it/s]
2021-03-24 06:27:04 - Re-ranking the top-k results ...
2021-03-24 06:27:06 - Generating BERT top-k results ...
2021-03-24 06:27:08 - Use pytorch device: cuda
100%|██████████| 1183/1183 [16:58<00:00,  1.16it/s]
2021-03-24 06:44:10 - Re-ranking the top-k results ...
2021-03-24 06:44:13 - Generating BERT top-k results ...
2021-03-24 06:44:15 - Use pytorch device: cuda
100%|██████████| 1183/1183 [16:03<00:00,  1.23it/s]
2021-03-24 07:00:20 - Re-ranking the top-k results ...
2021-03-24 07:00:23 - Generating BERT top-k results ...
2021-03-24 07:00:25 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:49<00:00,  1.25it/s]
2021-03-24 07:16:16 - Re-ranking the top-k results ...


**Re-ranking results using query_type="faq"; neg_type="simple"**

In [20]:
# define variables
query_type="faq"; neg_type="simple"; version="1.1"; loss_type='softmax'
bert_model_path='output/FAQIR/models/' + loss_type + '_' + neg_type + '_' + query_type + '_' + version

In [21]:
# create instance of ReRanker class
r = ReRanker(
    bert_model_path=bert_model_path, 
    test_queries=test_queries, relevance_label_df=relevance_label_df
)

In [22]:
# generate directory structure
reranked_output_path = output_path + "/supervised/" + loss_type + "/" + query_type + "/" + neg_type
make_dirs(reranked_output_path)

# next, generate BERT, Re-ranked top-k results and dump to files
bert_query_by_question = r.get_bert_topk_preds(es_query_by_question)
dump_to_json(bert_query_by_question, reranked_output_path + '/bert_query_by_question.json')
reranked_query_by_question = r.get_reranked_results(bert_query_by_question)
dump_to_json(reranked_query_by_question, reranked_output_path + '/reranked_query_by_question.json')

bert_query_by_answer = r.get_bert_topk_preds(es_query_by_answer)
dump_to_json(bert_query_by_answer, reranked_output_path + '/bert_query_by_answer.json')
reranked_query_by_answer = r.get_reranked_results(bert_query_by_answer)
dump_to_json(reranked_query_by_answer, reranked_output_path + '/reranked_query_by_answer.json')

bert_query_by_question_answer = r.get_bert_topk_preds(es_query_by_question_answer)
dump_to_json(bert_query_by_question_answer, reranked_output_path + '/bert_query_by_question_answer.json')
reranked_query_by_question_answer = r.get_reranked_results(bert_query_by_question_answer)
dump_to_json(reranked_query_by_question_answer, reranked_output_path + '/reranked_query_by_question_answer.json')

bert_query_by_question_answer_concat = r.get_bert_topk_preds(es_query_by_question_answer_concat)
dump_to_json(bert_query_by_question_answer_concat, reranked_output_path + '/bert_query_by_question_answer_concat.json')
reranked_query_by_question_answer_concat = r.get_reranked_results(bert_query_by_question_answer_concat)
dump_to_json(reranked_query_by_question_answer_concat, reranked_output_path + '/reranked_query_by_question_answer_concat.json')

2021-03-24 07:16:19 - Generating BERT top-k results ...
2021-03-24 07:16:31 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:11<00:00,  1.30it/s]
2021-03-24 07:31:45 - Re-ranking the top-k results ...
2021-03-24 07:31:47 - Generating BERT top-k results ...
2021-03-24 07:31:49 - Use pytorch device: cuda
100%|██████████| 1183/1183 [16:54<00:00,  1.17it/s]
2021-03-24 07:48:46 - Re-ranking the top-k results ...
2021-03-24 07:48:48 - Generating BERT top-k results ...
2021-03-24 07:48:50 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:56<00:00,  1.24it/s]
2021-03-24 08:04:49 - Re-ranking the top-k results ...
2021-03-24 08:04:51 - Generating BERT top-k results ...
2021-03-24 08:04:53 - Use pytorch device: cuda
100%|██████████| 1183/1183 [15:35<00:00,  1.26it/s]
2021-03-24 08:20:31 - Re-ranking the top-k results ...
